In [1]:
import torch
from torch.amp import autocast, GradScalers
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from tqdm import tqdm
from experiments.Checkpointing import save_checkpoint
from models.VAE import VAE
from modules.Losses import VAEVggLoss
from modules.SaveOutputs import save_reconstructions

torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()

dataset_dir: str = "./data/raw"
out_dir: str = "./outputs/"
batch_size: int = 64
latent_dim: int = 256
checkpoint_dir = "./experiments/checkpoints"
# epochs: int = 10
lr: float = 3e-4
beta: float = 0.5

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((64, 64)),  # resize to 224x224
    transforms.ToTensor()  # convert to tensor & scale to [0,1]
])

dataset = datasets.ImageFolder(root="G:\Temp", transform=transform)
train_test_split_var = 0.99
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(train_test_split_var*len(dataset)), len(dataset) - int(train_test_split_var*len(dataset))])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
n_train = len(train_loader.dataset)

print("Loaded datasets, number of samples: ", n_train)

# Model & Optimizer
scaler = GradScaler(device)
model = VAE(latent_dim=latent_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
vgg_loss = VAEVggLoss(recon_weight=10.0, perc_weight=0.001, kl_weight=1.0)

current_epoch = 0
# os.removedirs(out_dir)
os.makedirs(out_dir, exist_ok=True)


Loaded datasets, number of samples:  200573


D:\Software\Conda\envs\Torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Software\Conda\envs\Torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
def trainWithParams(epochs, variant, loss_function):
    print("Starting training... on variant {}...".format(variant))
    for epoch in tqdm(range(1, epochs + 1)):
        model.train()
        for batch_idx, (x, _) in enumerate(train_loader, start=1):
            x = x.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            logits, mean, logvar = model(x)
            loss, l1_loss, perc_loss, kl_loss = loss_function(logits, x, mean, logvar)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        save_reconstructions(model=model, x=x, out_dir=out_dir, step = epoch, device=device, variant=variant)

In [ ]:
train_for = 10

trainWithParams(train_for, "555M", VAEVggLoss(recon_weight=1.0, perc_weight=1.0, kl_weight=1.0, recon_loss_function = "mse"))
trainWithParams(train_for, "555L", VAEVggLoss(recon_weight=1.0, perc_weight=1.0, kl_weight=1.0, recon_loss_function = "l1"))
trainWithParams(train_for, "911M", VAEVggLoss(recon_weight=10.0, perc_weight=0.001, kl_weight=0.001, recon_loss_function = "mse"))
trainWithParams(train_for, "911L", VAEVggLoss(recon_weight=10.0, perc_weight=0.001, kl_weight=0.001, recon_loss_function = "l1"))
trainWithParams(train_for, "955M", VAEVggLoss(recon_weight=10.0, perc_weight=0.1, kl_weight=0.1, recon_loss_function = "mse"))
trainWithParams(train_for, "955L", VAEVggLoss(recon_weight=10.0, perc_weight=0.1, kl_weight=0.1, recon_loss_function = "l1"))
trainWithParams(train_for, "155M", VAEVggLoss(recon_weight=0.01, perc_weight=1.0, kl_weight=1.0, recon_loss_function = "mse"))
trainWithParams(train_for, "155L", VAEVggLoss(recon_weight=0.01, perc_weight=1.0, kl_weight=1.0, recon_loss_function = "l1"))
trainWithParams(train_for, "919M", VAEVggLoss(recon_weight=10, perc_weight=0.001, kl_weight=10, recon_loss_function = "mse"))
trainWithParams(train_for, "919L", VAEVggLoss(recon_weight=10, perc_weight=0.001, kl_weight=10, recon_loss_function = "l1"))
trainWithParams(train_for, "959M", VAEVggLoss(recon_weight=10, perc_weight=0.1, kl_weight=10, recon_loss_function = "mse"))
trainWithParams(train_for, "959L", VAEVggLoss(recon_weight=10, perc_weight=0.1, kl_weight=10, recon_loss_function = "l1"))

Starting training... on variant 555M...


100%|██████████| 10/10 [57:30<00:00, 345.08s/it]


Starting training... on variant 555L...


100%|██████████| 10/10 [57:23<00:00, 344.39s/it]


Starting training... on variant 911M...


100%|██████████| 10/10 [1:04:46<00:00, 388.65s/it]


Starting training... on variant 911L...


 40%|████      | 4/10 [30:27<45:41, 456.91s/it]  